In [ ]:
# Achei bem interessante a abordagem.
# Faltou um vinculo entre bicicleta e cliente. Além de não ser armazenado em nenhum lugar os dados digitados. Poderia ter usado uma lista para armazenar tudo que foi cadastrado.
# Vi também que é feito uma checagem de dados. Uma vez feito esse tipo de validação, o ideal é parar a execução uma vez que o dado é inválido. Rodando o código, se eu digitar um email inválido, o programa segue normalmente.
# E as fotos? Isso não foi pensado? Sei que não temos como trabalhar com isso, mas a ideia aqui era mais mostrar a organização do que o código pronto em si. Então coloquem isso na estrutura de vocês, ok?

In [7]:
#━━━━━━━━━❮Bibliotecas❯━━━━━━━━━
from datetime import datetime
import sys
import re
import urllib.request
import cv2a
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
# import pytesseract  #ele não instala corretamente, usar instalador para windows https://github.com/UB-Mannheim/tesseract/wiki
                    #instalado em C:\Program Files\Tesseract-OCR
#━━━━━━━━━━━━━━❮◆❯━━━━━━━━━━━━━━

ModuleNotFoundError: No module named 'cv2a'

In [12]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model

# Ativar a execução ansiosa
tf.config.run_functions_eagerly(True)

# Definir as pastas de imagens de bicicleta e não bicicleta
bicycle_dir = r'Sprint 01\com_bike'
not_bicycle_dir = r'Sprint 01\sem_bike'

# Carregar e pré-processar as imagens
data = []
labels = []

for image_file in os.listdir(bicycle_dir):
    image_path = os.path.join(bicycle_dir, image_file)
    image = load_img(image_path, target_size=(224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)

    data.append(image)
    labels.append('bicycle')

for image_file in os.listdir(not_bicycle_dir):
    image_path = os.path.join(not_bicycle_dir, image_file)
    image = load_img(image_path, target_size=(224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)

    data.append(image)
    labels.append('not_bicycle')

# Codificar os rótulos
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = tf.keras.utils.to_categorical(labels)

# Dividir os dados em conjuntos de treinamento e teste
(trainX, testX, trainY, testY) = train_test_split(np.array(data), np.array(labels), test_size=0.20, stratify=labels, random_state=42)

# Construir o gerador de imagens de treinamento para data augmentation
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15, horizontal_flip=True, fill_mode='nearest')

# Carregar a rede MobileNetV2, garantindo que os conjuntos de camadas FC sejam deixados de fora
baseModel = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# Construir a cabeça do modelo que será colocada em cima do modelo base
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name='flatten')(headModel)
headModel = Dense(128, activation='relu')(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation='softmax')(headModel)

# Colocar a cabeça do modelo no modelo base
model = Model(inputs=baseModel.input, outputs=headModel)

# Loop sobre todas as camadas no modelo base e congelá-las para que não sejam atualizadas durante o primeiro processo de treinamento
for layer in baseModel.layers:
    layer.trainable = False

# Configurar a programação de decaimento para a taxa de aprendizado
lr_schedule = ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=10000,
    decay_rate=0.9)

opt = Adam(learning_rate=lr_schedule)

model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'], run_eagerly=True)

# Treinar a cabeça do modelo
model.fit(aug.flow(trainX, trainY, batch_size=32), steps_per_epoch=len(trainX) // 32, validation_data=(testX, testY), validation_steps=len(testX) // 32, epochs=20)

# Salvar o modelo treinado
model.save('my_model.h5')

# Carregar o modelo treinado anteriormente
model = load_model('my_model.h5')

def preprocess_image(image_path):
    # Carregar a imagem e ajustá-la para o tamanho que o modelo espera
    image = load_img(image_path, target_size=(224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)
    return image

def classificar_imagem(caminho_imagem):
    # Pré-processar a imagem
    imagem = preprocess_image(caminho_imagem)

    # Fazer a previsão usando o modelo
    pred = model.predict(np.expand_dims(imagem, axis=0))

    # A previsão é um vetor de probabilidades. Vamos pegar a classe com a maior probabilidade
    classe_predita = np.argmax(pred, axis=1)

    # Retornar a classe predita. No nosso caso, 0 é "não bicicleta" e 1 é "bicicleta".
    if classe_predita == 0:
        print('Imagem invalida')
        sys.exit[0]
        return "não bicicleta"
    else:
        return "bicicleta"

caminho_imagem = 'Sprint 01\test_bike_image.jpg'
print(classificar_imagem(caminho_imagem))


FileNotFoundError: [WinError 3] O sistema não pode encontrar o caminho especificado: 'Sprint 01\\com_bike'

In [9]:
ano_atual = datetime.now().year

def mostrar_menu():
    print('━━━━━━━━━━━━━━━❮◆❯━━━━━━━━━━━━━━━━')
    print('1. Cadastrar bike ')
    print('2. Cadastrar cliente')
    print('3. Sair')
    print('━━━━━━━━━━━━━━━❮◆❯━━━━━━━━━━━━━━━━')

#Cadastro de bikes
def cadastro_bike():
    marca = input('Digite a marca da bike: ')
    print('')

    modelo = input('Digite o modelo da bike: ')
    print('')

    ano = int(input('Digite o ano de fabricação da bicicleta (entre 1900 e {}):'.format(ano_atual)))
    print('')

    foto =input('Digite o caminho da imagem: ')  #a ideia aqui é carregar criar um caminho para futuramente usar
    print('')                                    #um modelo de processamento de imagem que consiga detectar e classificar a bicicleta
                                                 # utilizando uma cnn, foi criado um modelo de exemplo, mas ele ainda não funciona por 
    # classificar_imagem(foto)
    print('Bike cadastrada com sucesso!')        #conta de um erro envolvendo os diretorios do meu computador, mas da pra ter uma ideia

    print('')

    if ano < 1900 or ano > ano_atual:
        print('Ano de fabricação inválido. Tente novamente.')
        print('')
        print('Por conta disso, código será interrompido')
        sys.exit(0)
    else:
         pass
        
    preco = float(input('Digite o preço da bike: '))
    print('')

    bicicleta = {
        'marca': marca,
        'modelo': modelo,
        'ano_fabricacao': ano,
        'preco': preco
    }

    print('Bicicleta cadastrada:')
    print('Marca:', bicicleta['marca'])
    print('')

    print('Modelo:', bicicleta['modelo'])
    print('')

    print('Ano de Fabricação:', bicicleta['ano_fabricacao'])
    print('')
    print('Preço: R$', bicicleta['preco'])



regex = '^[a-z0-9]+[\._]?[a-z0-9]+[@]\w+[.]\w{2,3}$'    
def check(email):  
    if(re.search(regex,email)):  
        print("Email Verificado com Sucesso")  
          
    else:  
        print("Email Invalido")  
        print('Por conta disso, código será interrompido')
        sys.exit(0)


# cadas
def cadastro_cliente():
    nome = input('Digite o nome do cliente: ')
    print('')

    sexo = input('Digite o sexo do cliente: ')
    print('')
        
    cpf = int(input('Digite o cpf do cliente: '))
    
    email =input('digite o email do cliente')
    check(email)
    print('')

    telefone = input('Digite o telefone do cliente: ')
    print('')

    ano_nas = int(input('Digite o ano de nascimento do cliente: '))
    print('')
    idade = ano_atual - ano_nas

    cep =input('digite o cep do cliente')
    print('')

    cliente = {
        'nome': nome,
        'sexo': sexo,
        'cpf': cpf,
        'email': email,
        'telefone': telefone,
        'idade': idade,
        'cep': cep
    }

    print('Cliente cadastrado:')
    print('Nome:', cliente['nome'])
    print('Sexo:', cliente['sexo'])
    print('CPF:', cliente['cpf'])
    print('Email:', cliente['email'])
    print('Telefone:', cliente['telefone'])
    print('Idade:', cliente['idade'])
    print('CEP:', cliente['cep'])











 # Loop para executar o programa até que o usuário escolha sair
while True:
    mostrar_menu()
    escolha = input('Escolha uma opção: ')
    
    if escolha == '1':
        cadastro_bike()

    elif escolha == '2':
        cadastro_cliente()

    elif escolha == '3':
        print('Obrigado por usar nosso programa de cadastro de bicicletas em Python. Até mais!')
        break

    else:
        print('Opção inválida. Por favor, escolha uma opção válida.')

#Code by Mhb


# Exibindo o dicionário

━━━━━━━━━━━━━━━❮◆❯━━━━━━━━━━━━━━━━
1. Cadastrar bike 
2. Cadastrar cliente
3. Sair
━━━━━━━━━━━━━━━❮◆❯━━━━━━━━━━━━━━━━


Email Verificado com Sucesso




Cliente cadastrado:
Nome: pedro
Sexo: femea
CPF: 42159315842
Email: maheborgas@gmail.com
Telefone: 11950602005
Idade: 18
CEP: 02404000
━━━━━━━━━━━━━━━❮◆❯━━━━━━━━━━━━━━━━
1. Cadastrar bike 
2. Cadastrar cliente
3. Sair
━━━━━━━━━━━━━━━❮◆❯━━━━━━━━━━━━━━━━


Email Invalido
Por conta disso, código será interrompido


SystemExit: 0

C:\Users\pytho\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
